In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
from sklearn import svm,neighbors
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate,KFold

from DataPrep import DATA
from ATLAS import ATLAS

from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.cross_decomposition import CCA

Atlas = ATLAS()

Data = DATA()
Data.Train_Test(0.8,12345)
Data.Add_MRI("brain")
Data.Split_Data()

In [ ]:
1+1

In [ ]:
#Do not run
def Train_Test_SVM_KCross(C_space,train,valid,train_index,valid_index):
    train_score=[]
    valid_score=[]
    for C in C_space:
        SVM = svm.SVC(kernel='linear',C=C)
        SVM.fit(train,Data.labels_train[train_index])
        train_score.append(SVM.score(train,Data.labels_train[train_index]))
        valid_score.append(SVM.score(valid,Data.labels_train[valid_index]))
        
    return(train_score,valid_score)

In [ ]:
#Do not run 

folds =4

kf = KFold(n_splits=folds,shuffle=True, random_state = 3)
C_space = np.logspace(-7,0,25)
comp=50

Train_temp = np.zeros((folds,C_space.shape[0]))
Valid_temp = np.zeros((folds,C_space.shape[0]))
i=0
idxs = [5,13,47]
for train_index, valid_index in kf.split(Data.features_train):
    pca = PCA(n_components=comp, whiten=False)
    pca.fit(Data.features_train[train_index,5:])
    X_train_pca = pca.transform(Data.features_train[train_index,5:])
    X_valid_pca = pca.transform(Data.features_train[valid_index,5:])

    train_pca = np.hstack((X_train_pca,Data.features_train[train_index,0:5]))
    valid_pca = np.hstack((X_valid_pca,Data.features_train[valid_index,0:5]))

    if len(idxs)==1:
        train = np.reshape(train_pca[:,idxs],(-1,1))
        valid = np.reshape(valid_pca[:,idxs],(-1,1))
    else:
        train = train_pca[:,idxs]
        valid = valid_pca[:,idxs]
    SVM_PCA = Train_Test_SVM_KCross(C_space,train,valid,train_index,valid_index)
    Train_temp[i,:] = SVM_PCA[0]
    Valid_temp[i,:] = SVM_PCA[1]
    i+=1
Train_temp = 
    
pca = PCA(n_components=comp, whiten=False)
pca.fit(Data.features_train[train_index,5:])


In [ ]:
#DO not run
folds =4

kf = KFold(n_splits=folds,shuffle=True, random_state = 3)
C_space = np.logspace(-7,1,25)
comp=50

idx_list = np.array([])
sweep = np.arange(0,train_pca.shape[1])
Train_Max = [] 
Valid_Max = []
Valid_STD = []
Test_Max = []
for i in range(10):   
    Train_Acc = []
    Valid_Acc = []
    Valid_std = []
    Test_Acc = []
    for idx in sweep:
        idxs = np.hstack((idx_list,idx)).astype('int')
        i=0
        Train_temp = np.zeros((folds,C_space.shape[0]))
        Valid_temp = np.zeros((folds,C_space.shape[0]))
        for train_index, valid_index in kf.split(Data.features_train):
            pca = PCA(n_components=comp, whiten=False)
            pca.fit(Data.features_train[train_index,5:])
            X_train_pca = pca.transform(Data.features_train[train_index,5:])
            X_valid_pca = pca.transform(Data.features_train[valid_index,5:])

            train_pca = np.hstack((X_train_pca,Data.features_train[train_index,0:5]))
            valid_pca = np.hstack((X_valid_pca,Data.features_train[valid_index,0:5]))

            if len(idxs)==1:
                train = np.reshape(train_pca[:,idxs],(-1,1))
                valid = np.reshape(valid_pca[:,idxs],(-1,1))
            else:
                train = train_pca[:,idxs]
                valid = valid_pca[:,idxs]
            SVM_PCA = Train_Test_SVM_KCross(C_space,train,valid,train_index,valid_index)
            Train_temp[i,:] = SVM_PCA[0]
            Valid_temp[i,:] = SVM_PCA[1]
            i+=1
        Train_mean = np.mean(Train_temp,axis=0)
        Valid_mean = np.mean(Valid_temp,axis=0)
        Valid_stds = np.std(Valid_temp,axis=0)
        
        Train_Acc.append(Train_mean[Valid_mean.argmax()])
        Valid_Acc.append(Valid_mean[Valid_mean.argmax()])
        Valid_std.append(Valid_stds[Valid_mean.argmax()])
 #       Test_Acc.append(SVM_PCA[4])
        
    Train_Acc = np.array(Train_Acc)
    Valid_Acc = np.array(Valid_Acc)
    idx_list = np.hstack((idx_list,sweep[Valid_Acc.argmax()]))
    sweep = np.delete(sweep, Valid_Acc.argmax())
    Train_Max.append(Train_Acc[Valid_Acc.argmax()])
    Valid_Max.append(max(Valid_Acc))
    Valid_STD.append(Valid_std[Valid_Acc.argmax()])
#    Test_Max.append(Test_Acc[Valid_Acc.argmax()])
    print(idx_list)

In [ ]:
# Do not run

xrange = np.arange(1,31)

plt.errorbar(x = xrange, y=Valid_Max,yerr=Valid_STD)
plt.plot(xrange,Test_Max,'.')
plt.plot(xrange,Train_Max,'--')
plt.xlabel("Components")
plt.ylabel("Accuracy")
plt.legend(["Test","Train (Avg)","Valid"],loc='lower right')
plt.title("Forward Step Selection PCA")
# plt.savefig("ForwardStep.pdf")

In [ ]:
from nilearn import masking

def Plot_EigenBrain(slices,minV,maxV):
    fig,axes= plt.subplots(ncols=3)
    axes[0].imshow(np.rot90(slices[0]), vmin=minV, vmax=maxV,cmap='seismic')
    axes[0].set_title("Sagittal")
    
    axes[1].imshow(np.rot90(slices[1]), vmin=minV, vmax=maxV,cmap='seismic')
    axes[1].set_title("Axial")
    
    a = axes[2].imshow(slices[2], vmin=minV, vmax=maxV,cmap='seismic')
    axes[2].set_title("Coronal")
    
    cbar = fig.colorbar(a,ax=axes[2], shrink=0.4)
    tick_locator = ticker.MaxNLocator(nbins=4)
    cbar.locator = tick_locator
    cbar.ax.yaxis.set_major_locator(ticker.AutoLocator())
    cbar.update_ticks()
    
    axes[0].axis("off")
    axes[1].axis("off")
    axes[2].axis("off")
    
    return fig

In [ ]:
Atlas.Print_ROIs()

In [ ]:
comp = 60

pca = PCA(n_components=comp, whiten=False)
pca.fit(Data.features_train[:,5:])

X_train_pca = pca.transform(Data.features_train[:,5:])
X_test_pca = pca.transform(Data.features_test[:,5:])

train_pca = np.hstack((X_train_pca,Data.features_train[:,0:5]))
test_pca = np.hstack((X_test_pca,Data.features_test[:,0:5]))

#Most predictive features (5. 13. 47. 31. 45.)

In [ ]:
from nilearn import plotting

plotting.view_img(masking.unmask(pca.components_[49], Data.masker))

In [ ]:
plotting.view_img(Atlas.Mask([37]))

In [ ]:
Atlas.Print_ROIs()

In [ ]:
print_slices = [9,5,49] #[5,13,47,31] 
for sliver in print_slices:
    slicer = masking.unmask(pca.components_[sliver], Data.masker).get_fdata()
    slices = [0,0,0]
    slices[0] = slicer[45,:,:]
    slices[1] = slicer[:,54,:]
    slices[2] = slicer[:,:,45]
    a = Plot_EigenBrain(slices,np.min(slicer),np.max(slicer))
    a.savefig("Video/PCA"+str(sliver)+".pdf")

In [ ]:
Data.idx_AD.shape

In [ ]:
Data.idx_CN.shape

In [ ]:
def Train_Test_SVM(train,test):
#     print("Training SVM...")
    train_score=[]
    valid_score=[]
    std_valid=[]
    C_space = np.logspace(-7,0,25)
    for C in C_space:
        SVM = svm.SVC(kernel='linear',C=C)
        cvs = cross_validate(SVM,train,Data.labels_train, cv=4,return_train_score=True)
        train_score.append(np.mean(cvs["train_score"]))
        valid_score.append(np.mean(cvs["test_score"]))
        std_valid.append(np.std(cvs["test_score"]))

    max_idx = valid_score.index(max(valid_score))
    maxC = C_space[max_idx]
    
#     if maxC == C_space[0] or maxC == C_space[-1]:
#         print("Error: C_range limit hit")
#         print(maxC)

    SVM = svm.SVC(kernel='linear',C=maxC)
    SVM.fit(train,Data.labels_train)
    test_acc = SVM.score(test,Data.labels_test)

    return(maxC,train_score[max_idx],valid_score[max_idx],std_valid[max_idx],test_acc)

In [ ]:
idx_list = np.array([])
sweep = np.arange(0,train_pca.shape[1])
Train_Max = [] 
Valid_Max = []
Valid_STD = []
Test_Max = []
for i in range(50):   
    Train_Acc = []
    Valid_Acc = []
    Valid_std = []
    Test_Acc = []
    for idx in sweep:
        idxs = np.hstack((idx_list,idx)).astype('int')
        if len(idxs)==1:
            train = np.reshape(train_pca[:,idxs],(-1,1))
            test = np.reshape(test_pca[:,idxs],(-1,1))
        else:
            train = train_pca[:,idxs]
            test = test_pca[:,idxs]
        SVM_PCA = Train_Test_SVM(train,test)
        Train_Acc.append(SVM_PCA[1])
        Valid_Acc.append(SVM_PCA[2])
        Valid_std.append(SVM_PCA[3])
        Test_Acc.append(SVM_PCA[4])
        
    Train_Acc = np.array(Train_Acc)
    Valid_Acc = np.array(Valid_Acc)
    idx_list = np.hstack((idx_list,sweep[Valid_Acc.argmax()]))
    sweep = np.delete(sweep, Valid_Acc.argmax())
    Train_Max.append(Train_Acc[Valid_Acc.argmax()])
    Valid_Max.append(max(Valid_Acc))
    Valid_STD.append(Valid_std[Valid_Acc.argmax()])
    Test_Max.append(Test_Acc[Valid_Acc.argmax()])
    print(idx_list)

In [ ]:
#18 per row
xrange = np.arange(1,51)

plt.errorbar(x = xrange, y=Valid_Max,yerr=Valid_STD)
plt.plot(xrange,Test_Max,'.')
plt.plot(xrange,Train_Max,'--')
plt.xlabel("Components")
plt.ylabel("Accuracy")
plt.legend(["Test","Train (Avg)","Valid"],loc='lower right')
# plt.title("Forward Step Selection PCA")
plt.savefig("ForwardStep.pdf")

In [ ]:
max(Test_Max)

In [ ]:
Test_Max[Valid_Max.index(max(Valid_Max))]

In [ ]:
max(Valid_Max)

In [ ]:
X_train_pca = pca.transform(Data.features_train[:,5:])
X_test_pca = pca.transform(Data.features_test[:,5:])

train_pca = np.hstack((X_train_pca,Data.features_train[:,0:5]))
test_pca = np.hstack((X_test_pca,Data.features_test[:,0:5]))

print("Training SVM...")
train_score_pca=[]
train_score_ica=[]
test_score_pca=[]
test_score_ica=[]
C_space = np.logspace(-4,15,30)
for C in C_space:
    SVM = svm.LinearSVC(penalty='l2',C=C,max_iter=1000000,dual=False)
    cvs = cross_validate(SVM,train_pca,Data.labels_train, cv=4,return_train_score=True)
    train_score_pca.append(np.mean(cvs["train_score"]))
    test_score_pca.append(np.mean(cvs["test_score"]))
    
    cvs = cross_validate(SVM,train_ica,Data.labels_train, cv=4,return_train_score=True)
    train_score_ica.append(np.mean(cvs["train_score"]))
    test_score_ica.append(np.mean(cvs["test_score"]))

maxC_pca = C_space[test_score_pca.index(max(test_score_pca))]
maxC_ica = C_space[test_score_ica.index(max(test_score_ica))]

plt.semilogx(C_space,test_score_pca)
plt.plot(C_space,test_score_ica)
plt.xlabel("C")
plt.ylabel("Accuracy")
plt.legend(["PCA","ICA"])

print("Training NN...")
train_score_pca=[]
train_score_ica=[]
test_score_pca=[]
test_score_ica=[]
neighbor_space = np.arange(1,5)
for neigh in neighbor_space:
    NN = neighbors.KNeighborsClassifier(neigh)
    cvs = cross_validate(NN,train_pca,Data.labels_train, cv=4,return_train_score=True)
    train_score_pca.append(np.mean(cvs["train_score"]))
    test_score_pca.append(np.mean(cvs["test_score"]))
    
    cvs = cross_validate(NN,train_ica,Data.labels_train, cv=4,return_train_score=True)
    train_score_ica.append(np.mean(cvs["train_score"]))
    test_score_ica.append(np.mean(cvs["test_score"]))

plt.figure()
plt.semilogx(neighbor_space,test_score_pca)
plt.plot(neighbor_space,test_score_ica)
plt.xlabel("Neighbors")
plt.ylabel("Accuracy")
plt.legend(["PCA","ICA"])

print("Training AdaBoost...")
train_score_pca=[]
train_score_ica=[]
test_score_pca=[]
test_score_ica=[]
neighbor_space = np.arange(1,5)
for neigh in neighbor_space:
    NN = neighbors.KNeighborsClassifier(neigh)
    cvs = cross_validate(NN,train_pca,Data.labels_train, cv=4,return_train_score=True)
    train_score_pca.append(np.mean(cvs["train_score"]))
    test_score_pca.append(np.mean(cvs["test_score"]))
    
    cvs = cross_validate(NN,train_ica,Data.labels_train, cv=4,return_train_score=True)
    train_score_ica.append(np.mean(cvs["train_score"]))
    test_score_ica.append(np.mean(cvs["test_score"]))

In [ ]:
x=np.array([1,2,3,4,5])
y=np.vstack((2.1*x,2*x,1.9*x))
yerr = np.std(y,axis=0)
plt.figure()
plt.errorbar(x,np.mean(y,axis=0),yerr)
plt.xscale("log", nonposx='clip')

In [ ]:
yerr

In [ ]:
#L1
print(max(test_score_pca),max(test_score_ica))

In [ ]:
#L2
print(max(test_score_pca),max(test_score_ica))

In [ ]:
cca = CCA(n_components=comp)
cca.fit(Data.features_train[:,5:], Data.labels_train)
X_train_cca = cca.transform(Data.features_train[:,5:])
X_test_cca = cca.transform(Data.features_test[:,5:])

In [ ]:
train_score_cca=[]
test_score_cca=[]

C_space = np.logspace(-4,15,30)
for C in C_space:
    SVM = svm.LinearSVC(penalty='l2',C=C,max_iter=1000000,dual=False)
    cvs = cross_validate(SVM,train_cca,Data.labels_train, cv=4,return_train_score=True)
    train_score_cca.append(np.mean(cvs["train_score"]))
    test_score_cca.append(np.mean(cvs["test_score"]))
    


In [ ]:
Data.features_train[:,5:].shape

In [ ]:
PCA_space = np.arange(1,25)*6
Train_Best = []
Valid_Best = []
Test_Best = []
MaxCs = []

for comp in PCA_space:
    pca = PCA(n_components=comp)
    pca.fit(Data.features_train[:,5:])

    X_train_pca = pca.transform(Data.features_train[:,5:])
    X_test_pca = pca.transform(Data.features_test[:,5:])

    train = np.hstack((X_train_pca,Data.features_train[:,0:5]))
    test = np.hstack((X_test_pca,Data.features_test[:,0:5]))

    print("Training....")
    train_score = []
    test_score = []
    C_space = np.logspace(-7,5,30)
    for C in C_space:
        SVM = svm.SVC(kernel='poly',degree=1, C=C)
        cvs = cross_validate(SVM,train,Data.labels_train, cv=4,return_train_score=True)
        train_score.append(np.mean(cvs["train_score"]))
        test_score.append(np.mean(cvs["test_score"]))
    #     print("C="+str(C)+"Train"+str(np.mean(cvs["train_score"]))+"Valid"+str(np.mean(cvs["test_score"])))

    maxC = C_space[test_score.index(max(test_score))]

    SVM = svm.SVC(kernel='linear', C=maxC)
    SVM.fit(train,Data.labels_train)
    pred_test=SVM.predict(test)

    Train_Best.append(train_score[test_score.index(max(test_score))])
    Valid_Best.append(max(test_score))
    Test_Best.append(accuracy_score(pred_test,Data.labels_test))
    MaxCs.append(maxC)

    print("PCA = "+str(comp))
    print("Max Acc = "+str(np.around(max(test_score),4))+", with C = "+str(maxC)+" Test Acc = "+str(accuracy_score(pred_test,Data.labels_test)))

plt.plot(PCA_space,Train_Best)
plt.plot(PCA_space,Valid_Best)
plt.plot(PCA_space,Test_Best)
plt.xlabel("PCA components")
plt.ylabel("Accuracy")
plt.legend(["Train","Valid","Test"])

In [ ]:
PCA_space = np.arange(1,15)*2
Train_Best = []
Valid_Best = []
Test_Best = []
MaxCs = []

for comp in PCA_space:
    pca = PCA(n_components=comp)
    pca.fit(Data.features_train[:,5:])

    X_train_pca = pca.transform(Data.features_train[:,5:])
    X_test_pca = pca.transform(Data.features_test[:,5:])

    train = np.hstack((X_train_pca,Data.features_train[:,0:5]))
    test = np.hstack((X_test_pca,Data.features_test[:,0:5]))

    print("Training....")
    train_score = []
    test_score = []
    C_space = np.logspace(-7,3,30)
    for C in C_space:
        SVM = svm.SVC(kernel='linear',C=C)
        cvs = cross_validate(SVM,train,Data.labels_train, cv=4,return_train_score=True)
        train_score.append(np.mean(cvs["train_score"]))
        test_score.append(np.mean(cvs["test_score"]))
    #     print("C="+str(C)+"Train"+str(np.mean(cvs["train_score"]))+"Valid"+str(np.mean(cvs["test_score"])))

    maxC = C_space[test_score.index(max(test_score))]

    SVM = svm.SVC(kernel='linear', C=maxC)
    SVM.fit(train,Data.labels_train)
    pred_test=SVM.predict(test)

    Train_Best.append(train_score[test_score.index(max(test_score))])
    Valid_Best.append(max(test_score))
    Test_Best.append(accuracy_score(pred_test,Data.labels_test))
    MaxCs.append(maxC)

    print("PCA = "+str(comp))
    print("Max Acc = "+str(np.around(max(test_score),4))+", with C = "+str(maxC)+" Test Acc = "+str(accuracy_score(pred_test,Data.labels_test)))

plt.plot(PCA_space,Train_Best)
plt.plot(PCA_space,Valid_Best)
plt.plot(PCA_space,Test_Best)
plt.xlabel("PCA components")
plt.ylabel("Accuracy")
plt.legend(["Train","Valid","Test"])

In [ ]:
train = Data.features_train[:,0:5]
test = Data.features_test[:,0:5]

train_score = []
test_score = []
C_space = np.logspace(-7,3,30)
for C in C_space:
    SVM = svm.SVC(kernel='linear',degree=2, C=C)
    cvs = cross_validate(SVM,train,Data.labels_train, cv=4,return_train_score=True)
    train_score.append(np.mean(cvs["train_score"]))
    test_score.append(np.mean(cvs["test_score"]))
#     print("C="+str(C)+"Train"+str(np.mean(cvs["train_score"]))+"Valid"+str(np.mean(cvs["test_score"])))

maxC = C_space[test_score.index(max(test_score))]

SVM = svm.SVC(kernel='linear', C=maxC)
SVM.fit(train,Data.labels_train)
pred_test=SVM.predict(test)

Train_Best.append(train_score[test_score.index(max(test_score))])
Valid_Best.append(max(test_score))
Test_Best.append(accuracy_score(pred_test,Data.labels_test))
MaxCs.append(maxC)

print("Max Acc = "+str(np.around(max(test_score),4))+", with C = "+str(maxC)+" Test Acc = "+str(accuracy_score(pred_test,Data.labels_test)))



# SVM vs Ada comparison

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

FileDir = os.getcwd()+'/SVM_Outputs/'
file_object = open(FileDir+'SVM_Log.txt', "r")
Data_Loader = np.zeros((45,5))
for (i,line) in enumerate(file_object):
    split = line.split(',')
    Data_Loader[i,0]=split[0] #ATLAS
    Data_Loader[i,1]=split[1] #Features
    Data_Loader[i,2]=split[2] #C
    Data_Loader[i,3]=split[3] #Valid
    Data_Loader[i,4]=split[4] #Test

In [ ]:
FileDir = os.getcwd()+'/Ada_Outputs/'
file_object = open(FileDir+'Ada_Log.txt', "r")
Data_Loader_Ada = np.zeros((45,5))
for (i,line) in enumerate(file_object):
    split = line.split(',')
    Data_Loader_Ada[i,0]=split[0] #ATLAS
    Data_Loader_Ada[i,1]=split[1] #Features
    Data_Loader_Ada[i,2]=split[2] #C
    Data_Loader_Ada[i,3]=split[3] #Valid
    Data_Loader_Ada[i,4]=split[4] #Test

In [ ]:
plt.plot(Data_Loader[:,0],Data_Loader[:,3],'b')
plt.plot(Data_Loader[:,0],Data_Loader[:,4],'--b')
plt.plot(Data_Loader_Ada[:,0],Data_Loader_Ada[:,3],'r')
plt.plot(Data_Loader_Ada[:,0],Data_Loader_Ada[:,4],'--r')
plt.xlabel("Atlas ROI")
plt.ylabel("Accuracy")
plt.legend(["SVM Valid","SVM Test","Ada Valid","Ada Test"],loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
idxs = np.argsort(Data_Loader[:,3]+Data_Loader_Ada[:,3])
print(Data_Loader[idxs,0])

In [ ]:
idxs = np.argsort(Data_Loader[:,3])
print(Data_Loader[idxs,0])
print(Data_Loader[idxs,3])

In [ ]:
idxs = np.argsort(Data_Loader_Ada[:,3])
print(Data_Loader_Ada[idxs,0])

# SVM Ensemble

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import cross_validate

from DataPrep import DATA

FileDir = os.getcwd()+'/SVM_Outputs/'
file_object = open(FileDir+'SVM_Log.txt', "r")
Data_Loader = np.zeros((45,5))
for (i,line) in enumerate(file_object):
    split = line.split(',')
    Data_Loader[i,0]=split[0] #ATLAS
    Data_Loader[i,1]=split[1] #Features
    Data_Loader[i,2]=split[2] #C
    Data_Loader[i,3]=split[3] #Valid
    Data_Loader[i,4]=split[4] #Test

Data = DATA()
Data.Train_Test(0.8,12345)
#test on 3,8, 26 - DL 2,7,25

In [ ]:
Ensemble = [] 
for (i,DL) in enumerate(Data_Loader):
    if DL[3]>=0.58:
        Ensemble.append(i)

In [ ]:
Predictions = np.zeros((len(Ensemble),Data.idx_test.shape[0]))
for (i,Ens) in enumerate(Ensemble):
    Selector = Data_Loader[Ens][0]
    maxC = Data_Loader[Ens][2]
    
    Data.Add_MRI([Selector])
    Data.Split_Data()
    
    SVM = svm.SVC(kernel='linear', C=maxC)
    SVM.fit(Data.features_train,Data.labels_train)
    
    pred_test = SVM.predict(Data.features_test)
    acc = accuracy_score(pred_test,Data.labels_test)
    
    Predictions[i]=pred_test
    
    print(Selector,acc)

In [ ]:
accuracy_score(scipy.stats.mode(Predictions[:,:],axis=0)[0][0],Data.labels_test)

# View Brain (Video Prep)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn import svm,neighbors
from sklearn.model_selection import cross_validate

from DataPrep import DATA
from ATLAS import ATLAS

from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
Data = DATA()
Data.Train_Test(0.8,12345)
Data.Add_MRI("brain") #"brain" or [34]
Data.Split_Data()

In [ ]:
train = Data.features_train
test = Data.features_test

train_score=[]
valid_score=[]
std_valid=[]
#0.0004281332398719391 whole brain
C_space = np.logspace(-2,1,30)
C_space = [0.0004281332398719391]
for C in C_space:
    SVM = svm.SVC(kernel='linear',C=C)
    cvs = cross_validate(SVM,train,Data.labels_train, cv=4,return_train_score=True)
    train_score.append(np.mean(cvs["train_score"]))
    valid_score.append(np.mean(cvs["test_score"]))
    std_valid.append(np.std(cvs["test_score"]))
    print(C,np.mean(cvs["train_score"]),np.mean(cvs["test_score"]),np.std(cvs["test_score"]))

max_idx = valid_score.index(max(valid_score))
maxC = C_space[max_idx]

SVM = svm.SVC(kernel='linear',C=maxC)
SVM.fit(train,Data.labels_train)
test_acc = SVM.score(test,Data.labels_test)

In [ ]:
plt.errorbar(x = C_space, y=valid_score,yerr=std_valid)
# plt.plot(C_space,Test_Max,'.')
plt.semilogx(C_space,train_score,'-')
plt.xlabel("C")
plt.ylabel("Accuracy")
plt.legend(["Train (Avg)","Valid"],loc='lower right')
plt.title("Whole Brain") #SVM_34_Parahippocampal Gyrus, anterior division
plt.ylim([0.4,1.02])
print(test_acc)
# plt.savefig("Video/SVM_WholeBrain.pdf")

In [ ]:
max(valid_score)

In [ ]:
from nilearn import masking
from nilearn import plotting

a = masking.unmask(SVM.coef_[0,5:], Data.masker)
plotting.view_img(a)

In [ ]:
movie = a.get_fdata()
plt.imshow(np.rot90(movie[32,:,:]), vmin=-np.max(np.abs(movie)), vmax=np.max(np.abs(movie)),cmap='seismic')
plt.colorbar()
plt.axis("off")
plt.savefig("Video/SVM_Hippo34_Weights.pdf")

In [ ]:
movie.shape[2]

In [ ]:
from celluloid import Camera

movie = a.get_fdata()

fig,axes= plt.subplots(ncols=2)
camera = Camera(fig)
for i in range(movie.shape[2]):
    axes[0].imshow(movie[:,:,i], vmin=np.min(movie), vmax=np.max(movie),cmap='seismic')
    axs = axes[1].imshow(np.rot90(movie[i,:,:]), vmin=np.min(movie), vmax=np.max(movie),cmap='seismic')
    axes[0].axis("off")
    axes[1].axis("off")
    camera.snap()
animation = camera.animate()
animation.save('Video/SVM_BrainCoeff.mp4', fps=15);

In [ ]:
plt.imshow(movie[:,:,i], vmin=np.min(movie), vmax=np.max(movie),cmap='seismic')
cbar = plt.colorbar()
tick_locator = ticker.MaxNLocator(nbins=5)
cbar.locator = tick_locator
cbar.ax.yaxis.set_major_locator(ticker.AutoLocator())
cbar.update_ticks()
plt.savefig("Video/Colobar_for_BrainCoeff.pdf")

In [ ]:
from celluloid import Camera

movie = Atlas.mask_img.get_fdata()

fig,axes= plt.subplots(ncols=2)
camera = Camera(fig)
for i in range(shape[2]):
    axes[0].imshow(movie[:,:,i], vmin=np.min(movie), vmax=np.max(movie),cmap='seismic')
    axes[1].imshow(np.rot90(movie[i,:,:]), vmin=np.min(movie), vmax=np.max(movie),cmap='seismic')
#     plt.colorbar()
    axes[0].axis("off")
    axes[1].axis("off")
    camera.snap()
animation = camera.animate()
animation.save('Brain_Atlas2.mp4', fps=15);

In [ ]:
Atlas=ATLAS()
print(mask_img.shape)
plotting.view_img(Atlas.mask_img)

In [ ]:
slicer = Atlas.Mask([35]).get_fdata()
slices = [0,0,0]
slices[0] = slicer[45,:,:]
slices[1] = slicer[:,54,:]
slices[2] = slicer[:,:,45]
a = Plot_EigenBrain(slices,np.min(slicer),np.max(slicer))

In [ ]:
from celluloid import Camera
import nibabel as nib
gm = nib.load(Data.gm_imgs[0])
wm = nib.load(Data.wm_imgs[0])
movie = gm.get_fdata()#+wm.get_fdata()

fig,axes= plt.subplots(ncols=2)
camera = Camera(fig)
for i in range(shape[2]):
    axes[0].imshow(movie[:,:,i], vmin=np.min(movie), vmax=np.max(movie),cmap='gray')
    axes[1].imshow(np.rot90(movie[i,:,:]), vmin=np.min(movie), vmax=np.max(movie),cmap='gray')
#     plt.colorbar()
    axes[0].axis("off")
    axes[1].axis("off")
    camera.snap()
animation = camera.animate()
animation.save('GMScan.mp4', fps=15);

In [ ]:
Data.idx_CN.shape

In [ ]:
Data.labels_test.shape

In [ ]:
27/39